In [3]:
import numpy as np
from scipy.special import factorial
from scipy import misc
from scipy import integrate
import sympy as sp

In [4]:
x_sympy = sp.Symbol('x')
f_sympy = sp.cos(x_sympy) / (x_sympy + 1)

In [30]:
def f(x: np.ndarray):
    return np.cos(x) / (x + 1)

def f_prime2(x: np.ndarray):
    return (2 * ((x + 1) * np.sin(x) + np.cos(x))) / (x + 1)**3 - np.cos(x) / (x + 1)

def nth_derivative(expr, var, n):
    d_expr = sp.diff(expr, var, n)
    return sp.lambdify([var], d_expr, modules='numpy'), d_expr

def gaussian_quadrature(f, a, b, m): # чому нампай ане руками
    z, w = np.polynomial.legendre.leggauss(m) # що таке поліноми лежандра 
    # як обраховуються поліноми лежандра 
    # чому поліноми лежандра використовуються як ваги для квадратури 
    x = 0.5 * ((b - a) * z + b + a) 
    w = 0.5 * (b - a) * w # що таке ваги у квадратурі гауса 
    return np.sum(w * f(x)) 

def trapezoidal_rule(n, f, f_prime2, b, a):
    x = np.linspace(b, a, n+1)
    fx = f(x)
    h = (b - a) / n
    integral = h * (0.5 * fx[0] + np.sum(fx[1:-1]) + 0.5 * fx[-1])
    threoretical_error = ((b - a)**3 / (12 * n**2)) * np.max(np.abs(f_prime2(x)))
    return integral, threoretical_error

def trapezoidal_rule_error_estimate(f, a, b, m):
    x = np.linspace(a, b, 1000)
    
    x_sympy = sp.Symbol('x')
    f_sympy = sp.cos(x_sympy) / (x_sympy + 1)
    f2m_prime, f2m_prime_vis = nth_derivative(f_sympy, x_sympy, 2*m)

    print(f2m_prime_vis)
    f_2m = f2m_prime(x)
    
    max_f_2m = np.max(np.abs(f_2m))
    term1 = factorial(m)**4
    term2 = (b - a)**(2*m+1)
    term3 = (2*m+1) * factorial(2*m)**3
    term4 = max_f_2m
    
    result = (term1 * term2) / (term3) * term4
    return result

def gaussian_quadrature_error_estimate(f, a, b, m):
    z, w = np.polynomial.legendre.leggauss(m)
    # change please to one that in pdf 
    x = 0.5 * ((b - a) * z + b + a)

    f2m_prime = nth_derivative(f_sympy, x_sympy, 2*m)[0]
    f_2m = f2m_prime(x)
    max_f_2m = np.max(np.abs(f_2m))
    return ((factorial(m)**4 * (b - a)**(2*m+1)) / 
            ((2*m+1) * factorial(2*m)**3)) * max_f_2m

In [31]:
a, b = 0.8, 1.07
tolerance = 0.0001

for mtemp in range(1, 10):
    estimate = gaussian_quadrature_error_estimate(f, a, b, mtemp)
    if estimate < tolerance:
        print(f"m: {mtemp}, {estimate:0.10f}")
        m = mtemp
        break

print(f"""
Integral of {f_sympy} at ({a}, {b}) 
m of {m} with analytical error of {estimate:0.10f}

Gaussian quadrature
-> {gaussian_quadrature(f, a, b, m)}

actual error: 
-> {gaussian_quadrature(f, a, b, m) - integrate.quad(f, a, b)[0]:1.10f}
""")

m: 2, 0.0000001614

Integral of cos(x)/(x + 1) at (0.8, 1.07) 
m of 2 with analytical error of 0.0000001614

Gaussian quadrature
-> 0.08309376731972307

actual error: 
-> -0.0000001227



In [34]:
trapezoidal_rule_error_estimate(f, a, b, 2)

(-cos(x) + 2*sin(x)/(x + 1) + 2*cos(x)/(x + 1)**2)/(x + 1)


np.float64(0.0002416732885754714)

In [35]:
trapezoidal_rule(10, f, f_prime2, b, a)

(np.float64(0.08309861298170938), np.float64(4.833465771509429e-06))